# Python for Finance: Historical Volatility & Risk-Return Measures

we compute and track historical volatility over time.

In [1]:
## This is required for pandas_datareader on google colab - then you need to restart runtime
#!pip install --upgrade pandas_datareader

In [2]:
import datetime as dt
import pandas as pd
import numpy as np
import yfinance as yf
from pandas_datareader import data as pdr
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots
yf.pdr_override()
pyo.init_notebook_mode(connected=True)
pd.options.plotting.backend = 'plotly'

In [3]:
### Get stock data with yfinance

In [4]:
end = dt.datetime.now()
start = dt.datetime(2010,1,1)

df = yf.download(['^AXJO', 'CBA.AX','NAB.AX','STO.AX'], start, end)
Close = df.Close
Close.head()

[*********************100%%**********************]  4 of 4 completed


Ticker,CBA.AX,NAB.AX,STO.AX,^AXJO
Date,,,,
2010-01-04,54.574314,26.065140,12.445209,4876.299805
2010-01-05,55.399841,26.398088,12.628099,4924.299805
2010-01-06,55.678333,25.884398,12.723898,4921.399902
2010-01-07,55.141243,25.675116,12.575844,4899.399902
2010-01-08,55.857365,25.589499,12.445209,4912.100098


### Compute log returns

In [5]:
log_returns = np.log(df.Close/df.Close.shift(1)).dropna()
log_returns

Ticker,CBA.AX,NAB.AX,STO.AX,^AXJO
Date,,,,
2010-01-05,0.015013,0.012693,0.014589,0.009795
2010-01-06,0.005014,-0.019651,0.007558,-0.000589
2010-01-07,-0.009693,-0.008118,-0.011704,-0.004480
2010-01-08,0.012903,-0.003340,-0.010442,0.002589
2010-01-11,0.007274,0.012927,0.007668,0.007827
...,...,...,...,...
2024-01-19,0.007087,0.012546,0.009204,0.010117
2024-01-22,0.013329,0.011127,0.002614,0.007437
2024-01-23,0.007463,0.009440,-0.001306,0.005110


### Calculate daily standard deviation of returns

In [6]:
daily_std = log_returns.std()
daily_std

Ticker
CBA.AX    0.012862
NAB.AX    0.014074
STO.AX    0.023842
^AXJO     0.009639
dtype: float64

In [7]:
annualized_std = daily_std * np.sqrt(252)
annualized_std

Ticker
CBA.AX    0.204178
NAB.AX    0.223425
STO.AX    0.378480
^AXJO     0.153016
dtype: float64

### Plot histogram of log returns with annualized volatility

In [8]:
fig = make_subplots(rows=2, cols=2)

trace0 = go.Histogram(x=log_returns['CBA.AX'], name='CBA')
trace1 = go.Histogram(x=log_returns['NAB.AX'], name='NAB')
trace2 = go.Histogram(x=log_returns['STO.AX'], name='STO')
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 2, 1)

fig.update_layout(autosize = False, width=700, height=600, title='Frequency of log returns',
                  xaxis=dict(title='CBA Annualized Volatility: ' + str(np.round(annualized_std['CBA.AX']*100, 1))),
                  xaxis2=dict(title='NAB Annualized Volatility: ' + str(np.round(annualized_std['NAB.AX']*100, 1))),
                  xaxis3=dict(title='STO Annualized Volatility: ' + str(np.round(annualized_std['STO.AX']*100, 1))),
                 )

fig.show(renderer="colab")

In [9]:
TRADING_DAYS = 60
volatility = log_returns.rolling(window=TRADING_DAYS).std()*np.sqrt(TRADING_DAYS)
volatility.tail()

Ticker,CBA.AX,NAB.AX,STO.AX,^AXJO
Date,,,,
2024-01-19,0.060131,0.069019,0.108749,0.051407
2024-01-22,0.060273,0.068922,0.108331,0.051738
2024-01-23,0.060103,0.069052,0.108299,0.051280
2024-01-24,0.060897,0.068973,0.108267,0.051288
2024-01-25,0.057791,0.065952,0.106605,0.050451


In [10]:
volatility.plot().update_layout(autosize = False, width=600, height=300).show(renderer="colab")

### Sharpe ratio
The Sharpe ratio which was introduced in 1966 by Nobel laureate William F. Sharpe is a measure for calculating risk-adjusted return. The Sharpe ratio is the average return earned in excess of the risk-free rate per unit of volatility.

In [11]:
Rf = 0.02/255
sharpe_ratio = (log_returns.rolling(window=TRADING_DAYS).mean() - Rf)*TRADING_DAYS / volatility

In [12]:
sharpe_ratio.plot().update_layout(autosize = False, width=600, height=300).show(renderer="colab")

#### Sortino Ratio
The Sortino ratio is very similar to the Sharpe ratio, the only difference being that where the Sharpe ratio uses all the observations for calculating the standard deviation the Sortino ratio only considers the harmful variance.

In [13]:
sortino_vol = log_returns[log_returns<0].rolling(window=TRADING_DAYS, center=True, min_periods=10).std()*np.sqrt(TRADING_DAYS)
sortino_ratio = (log_returns.rolling(window=TRADING_DAYS).mean() - Rf)*TRADING_DAYS / sortino_vol

In [14]:
sortino_vol.plot().update_layout(autosize = False, width=600, height=300).show(renderer="colab")

In [15]:
sortino_ratio.plot().update_layout(autosize = False, width=600, height=300).show(renderer="colab")

### Modigliani ratio (M2 ratio)

The Modigliani ratio measures the returns of the portfolio, adjusted for the risk of the portfolio relative to that of some benchmark.

In [16]:
m2_ratio = pd.DataFrame()

benchmark_vol = volatility['^AXJO']
for c in log_returns.columns:
    if c != '^AXJO':
        m2_ratio[c] = (sharpe_ratio[c]*benchmark_vol/TRADING_DAYS + Rf)*TRADING_DAYS

In [17]:
m2_ratio.plot().update_layout(autosize = False, width=600, height=300).show(renderer="colab")

### Max Drawdown

Max drawdown quantifies the steepest decline from peak to trough observed for an investment. This is useful for a number of reasons, mainly the fact that it doesn't rely on the underlying returns being normally distributed.

In [18]:
def max_drawdown(returns):
    cumulative_returns = (returns+1).cumprod()
    peak = cumulative_returns.expanding(min_periods=1).max()
    drawdown = (cumulative_returns/peak)-1
    return drawdown.min()


returns = df.Close.pct_change()
max_drawdowns = returns.apply(max_drawdown, axis=0)
max_drawdowns*100

/tmp/ipykernel_13520/722799521.py:8: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.



Ticker
CBA.AX   -43.361732
NAB.AX   -63.126540
STO.AX   -82.731243
^AXJO    -36.530541
dtype: float64

### Calmar Ratio

Calmar ratio uses max drawdown in the denominator as opposed to standard deviation.

In [19]:
calmars = np.exp(log_returns.mean()*255)/abs(max_drawdowns)
calmars.plot.bar().update_layout(autosize = False, width=600, height=300).show(renderer="colab")